In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)

In [2]:
import numpy as np
import pandas as pd

# Define the data
data = {
    "Size (sq ft)": [850, 900, 1200, 1400, 1600, 1700, 1800, 2000, 2200, 2500],
    "Bedrooms": [2, 3, 3, 3, 3, 4, 4, 4, 5, 5],
    "Price ($)": [300000, 340000, 400000, 500000, 520000, 580000, 600000, 620000, 720000, 790000]
}

df = pd.DataFrame(data)

X = df[["Size (sq ft)", "Bedrooms"]].values
y = df["Price ($)"].values

X.shape, y.shape

((10, 2), (10,))

In [3]:
from sklearn.preprocessing import StandardScaler

X = X.astype(np.float64)
y = y.astype(np.float64)

# scaler_X = StandardScaler()
# X = scaler_X.fit_transform(X)
# scaler_y = StandardScaler()
# y = scaler_y.fit_transform(y.reshape(-1, 1)).ravel()  # type: ignore

In [4]:
from models_sklearn import Ensemble, MLP, FONN1, FONN2, TREENN1, TREENN2
from models import MLP as MLP_2, FONN1 as FONN1_2, FONN2 as FONN2_2, TREENN1 as TREENN1_2, TREENN2 as TREENN2_2

ensemble_param_grid = {
}

nn_param_grid = {
    'activation': ['tanh'],
    # 'max_iter': [1000, 100, 2000],
    'max_iter': [400],
    'learning_rate': ['constant'],
    # 'learning_rate': ['adaptive'],
    # 'learning_rate_init': [1e-1, 1e-2, 1e-3],
    # 'learning_rate_init': [1e1, 1e-2, 1e-3],
    'learning_rate_init': [1e-2],
    # 'early_stopping': [True],
    'early_stopping': [False],
    'n_iter_no_change': [10000]
}

models = {}

num_trees_input = 5
num_trees_hidden = 5
hidden_nodes = [5]
# hidden_nodes = [5, 10, 40, 100]
# hidden_nodes = [(5,), (10,), (40,), (100,)]

models['Tree'] = Ensemble(1)
for hn in hidden_nodes:
    if isinstance(hn, tuple):
        models[f'Ensemble {sum(hn)}'] = Ensemble(sum(hn))
        models[f'MLP {hn}'] = MLP(hn)
        models[f'FONN1 {num_trees_input} {hn}'] = FONN1(
            num_trees_input, (num_trees_input+hn[0], *hn[1:]))
        models[f'FONN2 {num_trees_hidden} {hn}'] = FONN2(
            num_trees_hidden, (*hn[:-1], num_trees_hidden+hn[-1]))
        models[f'TREENN1 {hn}'] = TREENN1((1+hn[0], *hn[1:]))
        models[f'TREENN2 {hn}'] = TREENN2((*hn[:-1], 1+hn[-1]))
    else:
        models[f'Ensemble {hn}'] = Ensemble(hn)
        models[f'MLP {hn}'] = MLP(hn)
        models[f'FONN1 {num_trees_input} {hn}'] = FONN1(
            num_trees_input, num_trees_input+hn)
        models[f'FONN2 {num_trees_hidden} {hn}'] = FONN2(
            hn, num_trees_hidden+hn)
        models[f'TREENN1 {hn}'] = TREENN1(1+hn)
        models[f'TREENN2 {hn}'] = TREENN2(1+hn)

nn_param_grid_2 = {
    'epochs': [400],
    # 'epochs': [100],
    # 'learning_rate': [1e1, 1e0, 1e-1, 1e-2],
    # 'learning_rate': [1e-2, 1e-3],
    'learning_rate': [1e-2],
}

input_dim = X.shape[1]
output_dim = 1

for hn in hidden_nodes:
    models[f'MLP_2 {hn}'] = MLP_2(input_dim, hn, output_dim)
    models[f'FONN1_2 {num_trees_input} {hn}'] = FONN1_2(
        input_dim, hn, output_dim, num_trees_input)
    models[f'FONN2_2 {num_trees_hidden} {hn}'] = FONN2_2(
        input_dim, hn, output_dim, num_trees_hidden)
    # models[f'TREENN1_2 {hn}'] = ((
    #     TREENN1_2(input_dim, hn, output_dim), nn_param_grid_2)
    # models[f'TREENN2_2 {hn}'] = ((
    #     TREENN2_2(input_dim, hn, output_dim), nn_param_grid_2)
    models[f'TREENN1_2 {hn}'] = FONN1_2(input_dim, hn, output_dim, 1)
    models[f'TREENN2_2 {hn}'] = FONN2_2(input_dim, hn, output_dim, 1)
    # models['Tree-based Predictions (FONN2)'] = models['FONN2'].trees
    # models['Tree-based Predictions (TREENN2)'] = models['TREENN2'].trees

In [5]:
# import sys
# import os
# from warnings import simplefilter
# if not sys.warnoptions:
#     simplefilter("ignore")
#     os.environ["PYTHONWARNINGS"] = "ignore"

# # Train and evaluate models
# for name, model in models.items():
#     model.fit(X, y)

In [6]:
mlp = MLP((10, 5))
mlp.fit(X, y)
len(mlp.coefs_), len(mlp.intercepts_), [c.shape for c in mlp.coefs_], [i.shape for i in mlp.intercepts_]

/home/chess/anaconda3/envs/research/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


(3, 3, [(2, 10), (10, 5), (5, 1)], [(10,), (5,), (1,)])

In [7]:
mlp.coefs_[-1]

array([[ 0.6675725 ],
       [-0.01740139],
       [ 1.20226013],
       [ 1.17515261],
       [ 0.95843878]])

In [8]:
mlp_2 = MLP_2(input_dim, (10, 5), output_dim)
len(mlp_2.coefs_), len(mlp_2.intercepts_), [c.shape for c in mlp_2.coefs_], [i.shape for i in mlp_2.intercepts_]

self.coefs_ = []
self.intercepts_ = []

hidden_dim = self.hidden_dim
if not hasattr(hidden_dim, "__iter__"):
    hidden_dim = [hidden_dim]
hidden_dim = list(hidden_dim)

layer_units = [self.input_dim] + hidden_dim + [self.output_dim]
self.n_layers_ = len(layer_units)

for i in range(self.n_layers_ - 1):
    coef_init = np.random.randn(layer_units[i], layer_units[i + 1])
    intercept_init = np.zeros(layer_units[i+1])
    self.coefs_.append(coef_init)
    self.intercepts_.append(intercept_init)

(3, 3, [(2, 10), (10, 5), (5, 1)], [(10,), (5,), (1,)])

In [9]:
mlp_2.coefs_[-1], mlp_2.intercepts_[-1]

(array([[ 0.17097097],
        [-2.41645177],
        [-0.53310392],
        [ 1.62438206],
        [ 0.32513365]]),
 array([0.]))

In [12]:
activations = mlp_2._forward(X)
[a.shape for a in activations]

[(10, 2), (10, 10), (10, 5), (10, 1)]

In [11]:
loss = activations[-1] - y

coef_grads = [
    np.empty((n_fan_in_, n_fan_out_), dtype=X.dtype)
    for n_fan_in_, n_fan_out_ in zip(mlp_2.layer_units[:-1], mlp_2.layer_units[1:])
]

intercept_grads = [
    np.empty(n_fan_out_, dtype=X.dtype) for n_fan_out_ in mlp_2.layer_units[1:]
]

# Compute gradients for the output layer
coef_grads[-1] = np.dot(activations[-2].T, loss) / y.shape[0]
intercept_grads[-1] = np.mean(loss, axis=0)
print(coef_grads[-1].shape)
# Compute the gradients for the hidden layers
for i in range(mlp_2.n_layers_ - 2, 0, -1):
    loss = np.dot(loss, mlp_2.coefs_[i].T)
    loss *= (1 - np.tanh(activations[i])**2)  # Tanh derivative
    coef_grads[i-1] = np.dot(activations[i-1].T, loss) / y.shape[0]
    intercept_grads[i-1] = np.mean(loss, axis=0)

# # Gradient clipping to prevent exploding gradients
# max_grad_norm = 1.0
# for i in range(self.n_layers_):
#     coef_grads[i] = np.clip(
#         coef_grads[i], -max_grad_norm, max_grad_norm)
#     intercept_grads[i] = np.clip(
#         intercept_grads[i], -max_grad_norm, max_grad_norm)

#     # Update weights and biases using gradient descent
#     self.coefs_[i] -= self.learning_rate * coef_grads[i]
#     self.intercepts_[i] -= self.learning_rate * intercept_grads[i]

(5, 10)


ValueError: shapes (10,10) and (1,5) not aligned: 10 (dim 1) != 1 (dim 0)